In [2]:
import pandas as pd
from data_pipeline import data_pipeline, restore_authors, dot_problem_solver
from math import ceil
from pathlib import Path

In [3]:
file_rzepa = 'Rzeczpospolita 2015-2022.xlsx'
data = pd.read_excel(file_rzepa)
drive_korpusy = Path("/run/user/1000/gvfs/google-drive:host=gmail.com,user=przemek.7678/Shared drives/SKN AI FUW /Projekt z Dagmarą Mateją/Korpusy/Rzeczpospolita 2015-2022")

In [8]:
cols = ['title', 'text', 'image_text']

In [9]:
data = data_pipeline(data, cols)
data = restore_authors(data)
data['text'] = data['text'].apply(dot_problem_solver)

/home/ps/Code/Climate/DataPrep/Rzepa/data_pipeline.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna('None')
/home/ps/Code/Climate/DataPrep/Rzepa/data_pipeline.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: re.split('(?i)©℗\s*Więcej\s*na', x)[0])
/home/ps/Code/Climate/DataPrep/Rzepa/data_pipeline.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [10]:
data.to_csv("rzepa_all.csv", index=False)

# chunks

In [ ]:
!mkdir chunks

In [ ]:
def chunkit(df, chunk_size= 10000):
    num_chunks = ceil(len(df) / chunk_size)
    for i in range(num_chunks):
        yield df[i*chunk_size:(i + 1) * chunk_size]

for i, chunk in enumerate(chunkit(data)): chunk.to_csv(f"chunks/rzepa_{i}.csv", index=False)

In [ ]:
!rm -r chunks